<a href="https://colab.research.google.com/github/manuel-alvarez/scheduling/blob/master/poc_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Generator with Genetic Algorithms

This script is intended to be a schedule generator for schools.

There are three type of resources:
 - Classrooms
 - Teachers
 - Subjects

For this script we will consider that all students in a class go together and don't move, so they are something like "attached to the classroom" and therefore not considered as resources, but they could.

## Description

In this version, we are going to consider two of the three types of resources as constant, and one of them as variable. For example:
 - 1 classroom
 - 1 teacher
 - Multiple subjects (where number of subjects is greater than segment length)

### Distribution

In this version, subjects were randomly distributed, but now we want the subjects to be distributed with sense. Some subject must have 5 hours per week, while other just have one. We will use a dictionary with distribution and we will compare this to the unique (with counts) method in get_fitness.

We also have distribution for teachers, and we have some teachers that can teach different subjects. Sometimes, a subject can be teached by several teachers.

### Constraints

In this example, we will use constraints. Sometimes, resources (subjects, teachers, ...) cannot be used in certain hours. This will be configured with an array of zeros and ones. 0 means it can be used, 1 means it cannot.

## Requirements

In [0]:
import numpy

## Settings

All this settings would be better in a database table, but we will use them as constants in those examples.

In [0]:
# Population (total size of time slots)
POPULATION_SIZE = 100
# Each segment represents a day
SEGMENT = 5
# Five days, since the schedule is weekly based, that are 5 segments
INDIVIDUAL_SIZE = 25
CELL_SIZE = 25
# Resources, currently, of just one type, let's say subjects
RESOURCES = 10
# Number of individuals that pass to the next generation
SURVIVAL_RATE = .3
# Rate of elements that are going to mutate
MUTATION_RATE = .1
# Rate of elements (in first population * rate positions) that are not mutating
STEADY_POPULATION = .1
# Rate of genes that are going to be mutate
MUTATIONS = .2
# Number of iterations we do in order to get the best approach
STEPS = 500
# Positions that must remain empty
EMPTY = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
# Severity
SEVERITY = {
    'high': 6,
    'medium': 3,
    'low': 1
}
# Distribution with examples for a primary school in Spain
# Array with two elements. 
#  1.- Distribution of time slots for each subject
#  2.- Distribution of subjects that can be teached by each teacher
DISTRIBUTION = [
  {
      1: 5,  # Spanish
      2: 4,  # Math
      3: 4,  # English
      4: 2,  # Social Science
      5: 2,  # Natural Science
      6: 2,  # PE
      7: 1,  # Religion
      8: 1,  # Arts & crafts
      9: 1,  # Music
      10: 1  # Local culture (Asturian)
  },
  {
      1: [1, 4, 7, 10],  # Alice
      2: [2, 5, 7],      # Bob
      3: [3, 6, 8],      # Claire
      4: [6],            # Dan
      5: [9]             # Elisabeth
  }
]
CONSTRAINTS = {
    1: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
    2: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
    3: None,
    4: None,
    5: None,
    6: None,
    7: None,
    8: None,
    9: None,
    10: None
}
SUBJECT_LABELS = {
    0: '',
    1: 'Spanish',
    2: 'Math',
    3: 'English',
    4: 'Social Science',
    5: 'Natural Science',
    6: 'PE',
    7: 'Religion',
    8: 'Arts & crafts',
    9: 'Music',
    10: 'Local culture',
}
TEACHER_LABELS = {
    0: '',
    1: 'Alice',
    2: 'Bob',
    3: 'Claire',
    4: 'Dan',
    5: 'Elisabeth',
}

## Methods

### create_individual

In [0]:
def teachers_by_subject():
  teachers = {0: [0]}
  for teacher in DISTRIBUTION[1].keys():
    for key in DISTRIBUTION[1][teacher]:
      teachers.setdefault(key, [])
      teachers[key].append(teacher)

  return teachers

In [0]:
def create_individual(cell_size):
  """
  Individuals are created with some constraints, so it's easier to find a valid
  one
  First half of each individual y subject oriented (timeslots with subjects), 
  last half is teacher oriented (timeslots with teachers).
  """
  individual = []
  # Fisrt half
  for key in DISTRIBUTION[0].keys():
    for i in range(DISTRIBUTION[0][key]):
      individual.append(key)
  numpy.random.shuffle(individual)
  for i in range(len(individual), cell_size):
    individual.append(0)

  # Last half
  # Random pick teachers
  teachers = teachers_by_subject()
  teachers = {subject: numpy.random.choice(teachers[subject]) for subject in teachers}
  last_half = []
  for i in individual:
    last_half.append(teachers[i])
  individual += last_half

  return individual  

In [0]:
numpy.reshape(create_individual(INDIVIDUAL_SIZE), (2, 5, 5))


### initialize_population

In [0]:
def initialize_population(population_size, cell_size):
  population = None
  for i in range(population_size):
    if population is None:
      population = numpy.array([create_individual(cell_size)])
    else:
      population = numpy.append(population, [create_individual(cell_size)], axis=0)
  return population

In [0]:
population = initialize_population(POPULATION_SIZE, CELL_SIZE)
population.shape

### get_fitness

In [0]:
def get_distribution(individual):
  unique, counts = numpy.unique(individual, return_counts=True)
  actual_distribution = dict(zip(unique, counts))
  return actual_distribution

In [0]:
def get_fitness(the_try):
  """
  Each individual has two halves we will call cells. First half is subject based
  and last half is teachers based. We will split each try in those two halves.
  """
  fitness = 0

  # Find fitness for first half
  first_half = numpy.reshape(the_try, (2, CELL_SIZE))[0]
  for i in range(int(numpy.ceil(CELL_SIZE / SEGMENT))):
    # print(f'Evaluating', i*SEGMENT, (i+1)*SEGMENT, first_half[i*SEGMENT:(i+1)*SEGMENT], 'from', first_half)
    unique, counts = numpy.unique(first_half[i*SEGMENT:(i+1)*SEGMENT], return_counts=True)
    # Not repeated resources in same segment
    # Except 0
    count = dict(zip(unique, counts))
    if 0 in count.keys():
      count.pop(0)
    fitness += sum(SEVERITY['medium'] for item in count.values() if item > 1)

  # fitness by distribution
  actual_distribution = get_distribution(first_half)
  fitness += sum(numpy.abs(actual_distribution.get(key, 0) - DISTRIBUTION[0][key]) for key in DISTRIBUTION[0].keys())
  # Some positions must remain empty
  fitness += sum(EMPTY[index] * SEVERITY['high'] * item / item for index, item in enumerate(first_half) if item != 0)
  # Not empty slots in try unless it's mandatory
  fitness += sum(SEVERITY['medium'] for index, item in enumerate(first_half) if item == 0 and EMPTY[index] == 0) 

  # Fitness by constraints
  for resource in CONSTRAINTS:
    if CONSTRAINTS[resource] is not None:
      items = numpy.array([1 if key == resource else 0 for key in first_half])
      constraints = numpy.array(CONSTRAINTS[resource])
      fitness += sum(items * constraints) * SEVERITY['medium']

  # At this time, the only constraints that has the second half is the 
  # subject->teacher constraint. As they are set in the creation stage, we don't
  # need to check them out already.

  return fitness

### get_selection

In [0]:
def get_selection(population):
  """
  First, remove duplicates
  Then leave just the SURVIVAL_RATE of the total population
  Finally sort them by fitness
  """
  selection = numpy.unique(population, axis=0)
  selection = [[get_fitness(the_try), the_try] for the_try in selection]
  selection = sorted(selection, key=lambda x:x[0])
  selection = selection[:int(numpy.ceil(POPULATION_SIZE * SURVIVAL_RATE))]
  selection = numpy.array(selection)
  return selection


### breed

In [0]:
def breed(selection):
  """
  Each individual has two halves that are related with each other. As they are
  directly related, we will split them in the same point and will mix them in 
  the same way
  """
  population = numpy.array([the_try for the_try in selection])
  i = 0
  while len(population) < POPULATION_SIZE:
    if i <= len(selection):
      parents = population[i:i+2]
      numpy.random.shuffle(parents)
      slicery = numpy.random.randint(CELL_SIZE)
      first_half_0, last_half_0 = numpy.reshape(parents[0], (2, CELL_SIZE))
      first_half_1, last_half_1 = numpy.reshape(parents[1], (2, CELL_SIZE))
      new_individual = numpy.concatenate((first_half_0[:slicery], first_half_1[slicery:], last_half_0[:slicery], last_half_1[slicery:]), axis=0)
      population = numpy.append(population, numpy.array([new_individual]), axis=0)
      i += 1
    # Once reached the end, create new random individuals
    else:
      population = numpy.append(population, [create_individual(CELL_SIZE)], axis=0)

  return population

In [0]:
# POPULATION_SIZE = 5
# CELL_SIZE = 25
population = initialize_population(3, CELL_SIZE)
breed(population)

### mutate

In [0]:
def mutate(population):
  mutated_population = population.copy()
  mutants = int(numpy.ceil(POPULATION_SIZE * MUTATION_RATE))
  for i in range(mutants):
    index = numpy.random.randint(POPULATION_SIZE * STEADY_POPULATION, POPULATION_SIZE)
    # Do NOT mutate accurate individuals
    if get_fitness(mutated_population[index]) > 0:
      mutant = numpy.reshape(mutated_population[index], (2, CELL_SIZE))
      num_mutations = int(numpy.ceil(CELL_SIZE) * MUTATIONS)
      for mutation in range(num_mutations):
        gene_index_0 = numpy.random.randint(CELL_SIZE)
        gene_index_1 = numpy.random.randint(CELL_SIZE)
        tmp_f = mutant[0][gene_index_0]
        tmp_l = mutant[1][gene_index_0]
        mutant[0][gene_index_0] = mutant[0][gene_index_1]
        mutant[1][gene_index_0] = mutant[1][gene_index_1]
        mutant[0][gene_index_1] = tmp_f
        mutant[1][gene_index_1] = tmp_l
      mutated_population[index] = mutant.flatten()

  return mutated_population

### print_calendar

In [0]:
def print_calendar(individual):
  subjects, teachers = numpy.reshape(individual, (2, CELL_SIZE))
  calendar = ['%8s, %5s' % (SUBJECT_LABELS[int(subjects[int(i)])][:8], TEACHER_LABELS[int(teachers[int(i)])][:5]) for i, subject in enumerate(subjects)]
  print(numpy.reshape(calendar, (SEGMENT, SEGMENT)))

## Main script

In [186]:
stats = []
for times in range(1):
  population = initialize_population(POPULATION_SIZE, CELL_SIZE)
  for i in range(STEPS):
    # print(f'Step {i}')
    selection = get_selection(population)
    # print(selection[0:3])
    if all([item[0] == 0 for item in selection[0:3]]):
      stats.append(i)
      print(times + 1)
      break
    population = breed(selection[:,1])
    population = mutate(population)

for i in range(3):
  print_calendar(selection[:,1][i])
  # print(get_distribution(selection[:,1][i]))

print(stats)
print('times', len(stats))
print('max', numpy.max(stats))
print('min', numpy.min(stats))
print('average', numpy.average(stats))
print('median', numpy.median(stats))

1
[[' English, Clair' 'Social S, Alice' ' Spanish, Alice' 'Natural ,   Bob'
  '    Math,   Bob']
 [' English, Clair' 'Arts & c, Clair' ' Spanish, Alice' '   Music, Elisa'
  '    Math,   Bob']
 ['      PE, Clair' ' English, Clair' ' Spanish, Alice' '    Math,   Bob'
  'Social S, Alice']
 ['Religion,   Bob' '    Math,   Bob' ' Spanish, Alice' ' English, Clair'
  'Local cu, Alice']
 ['      PE, Clair' 'Natural ,   Bob' ' Spanish, Alice' '        ,      '
  '        ,      ']]
[[' English, Clair' 'Social S, Alice' '    Math,   Bob' 'Natural ,   Bob'
  ' Spanish, Alice']
 [' English, Clair' 'Arts & c, Clair' ' Spanish, Alice' '    Math,   Bob'
  'Natural ,   Bob']
 [' English, Clair' '      PE, Clair' ' Spanish, Alice' '    Math,   Bob'
  'Social S, Alice']
 ['Religion,   Bob' '    Math,   Bob' ' Spanish, Alice' '   Music, Elisa'
  ' English, Clair']
 ['      PE, Clair' ' Spanish, Alice' 'Local cu, Alice' '        ,      '
  '        ,      ']]
[[' English, Clair' 'Social S, Alice' '    Mat

## Stats stored for comparison

In [112]:
all_stats = [
  [33, 86, 233, 41, 65, 64, 155, 74, 118, 101, 115, 109, 71, 85, 70, 64, 184, 75, 232, 53, 82, 26, 238, 125, 76, 67, 83, 15, 222, 78, 211, 236, 187, 124, 106, 84, 199, 64, 110, 180, 26, 114, 90, 239, 69, 217, 49, 143, 164, 94, 135, 37, 152, 134, 123, 57, 55, 125, 25, 182, 176, 144, 56, 135, 96, 76, 31, 9, 163, 46, 24, 268, 175, 43, 125, 56, 95, 49, 128, 93, 205, 216, 73, 240, 178, 142, 127, 122, 342, 88, 109, 71, 150, 162, 86, 70, 94, 119, 266, 55],
  [56, 47, 89, 62, 67, 102, 77, 137, 54, 61, 199, 53, 80, 30, 36, 175, 44, 42, 45, 26, 91, 99, 54, 99, 49, 103, 188, 10, 216, 147, 265, 201, 180, 90, 114, 264, 166, 88, 142, 96, 168, 49, 276, 114, 177, 280, 54, 134, 127, 207, 94, 107, 180, 90, 171, 186, 145, 77, 106, 212, 87, 102, 96, 67, 91, 118, 24, 12, 52, 120, 41, 113, 122, 161, 102, 43, 91, 73, 59, 110, 54, 93, 178, 132, 140, 141, 135, 131, 103, 84, 118, 160, 404, 304, 97, 175, 78, 127, 58, 55],
  [106, 93, 61, 28, 19, 41, 67, 26, 57, 45, 41, 83, 23, 37, 22, 28, 43, 109, 21, 190, 50, 29, 32, 34, 68, 35, 32, 61, 227, 126, 61, 66, 35, 41, 78, 102, 37, 68, 66, 99, 54, 55, 42, 34, 25, 44, 23, 35, 36, 53, 48, 60, 42, 66, 34, 37, 86, 48, 99, 39, 66, 19, 79, 30, 77, 31, 30, 42, 47, 37, 55, 87, 25, 42, 43, 35, 69, 34, 42, 128, 63, 33, 49, 50, 49, 33, 52, 51, 38, 57, 110, 90, 40, 41, 65, 67, 37, 58, 71, 60],
]
for stats in all_stats:
  print('--------------------------')
  print('times', len(stats))
  print('max', numpy.max(stats))
  print('min', numpy.min(stats))
  print('average', numpy.average(stats))
  print('median', numpy.median(stats))

--------------------------
times 100
max 342
min 9
average 116.74
median 103.5
--------------------------
times 100
max 404
min 10
average 115.49
median 102.0
--------------------------
times 100
max 227
min 19
average 55.79
median 47.5
